Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [67]:
from random import choices, choice, randint, random
from dataclasses import dataclass
from copy import copy, deepcopy

import lab9_lib

In [4]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

00101001101000010011100010001100101110000100010100: 17.56%
00010011000001000000000110001010000110111100001101: 11.56%
01100111010111001110011100100110110011011100100000: 23.33%
00011010101111010000110000101001000011010010110010: 17.56%
11100001100110111100101010011100100011011000110110: 9.13%
00010011010001011011000001111110101100000010010010: 7.33%
00010000110010101101101101111111111000100010110000: 7.33%
10000111011100001111101111000110111001101011001110: 9.13%
00110110110011101011000010000001001110011110100001: 15.34%
01101000001110111001111011110110101100111100111011: 9.11%
10


In [328]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = 0.15

NUM_SETS = 50

@dataclass
class Individual:
    fitness: float
    genotype: list[int]

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual, population) -> Individual:
    offspring = copy(ind)

    best = NUM_SETS
    index = -1
    for i in range(NUM_SETS):
        counter = 0
        for j in range(len(population)):
            if population[j].genotype[i] == 1:
                counter += 1

        if counter < best:
            best = counter
            index = i

    if index != -1:
        pos = index
    else:
        pos = randint(0, NUM_SETS-1)

    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    list = [0 for _ in range(NUM_SETS)]

    for i in range(NUM_SETS):
        list[i] = ind1.genotype[i] or ind2.genotype[i]

    offspring = Individual(fitness=None,
                           genotype=list)
    assert len(offspring.genotype) == NUM_SETS
    return offspring

In [158]:
def populate():
    population = [
        Individual(
            genotype=[choice((0,1)) for _ in range(NUM_SETS)],
            fitness=None,
        )
        for _ in range(POPULATION_SIZE)
    ]

    for i in population:
        i.fitness = fitness(i.genotype)

    return population

In [332]:
fitness = lab9_lib.make_problem(10)

population = populate()
for generation in range(3):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:
            p = select_parent(population)
            o = mutate(p, population)
        else:
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    print(f"{population[0].fitness:.2%}")

print(fitness.calls)

47.33%
89.20%
100.00%
90


In [269]:
μ = 10
λ = 20 #number of children
MUTATION_RATE = 0.5
GENERATION_NUM = 5_00 // λ

def es():
    fitness = lab9_lib.make_problem(10)

    population = populate()

    population.sort(key = lambda x:x.fitness, reverse = True)

    parents = population[:μ]

    for _ in range(GENERATION_NUM): #NEW GENERATION
        children = []

        for p in parents:
            children.append(deepcopy(p))

        for _ in range(λ):
            parent = choice(parents)
            new_i = mutate(parent, children)
            new_i.fitness = fitness(new_i.genotype)

            children.append(new_i)

        population = deepcopy(children)

        population.sort(key = lambda x:x.fitness, reverse = True)
        
        parents = population[:μ]
        print(f"{population[0].fitness:.2%}")

    print(fitness.calls)

In [273]:
es()

35.58%
35.58%
41.78%
41.78%
47.33%
47.36%
47.36%
55.34%
55.34%
69.11%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
500
